In [3]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import contractions

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir) #reading the data directory to list all the files
file_paths = [(data_dir + '/' + fname) for fname in file_names] #forming file paths
docID_to_doc_mapping = {} #forming docID to doc name mapping
for i in range(len(file_names)):
    docID_to_doc_mapping[i] = file_names[i]

In [5]:

def read_files(fpaths):
    '''
        Reads the files and pre process every file's text to form word tokens for every file.
        Returns a 2-D list containing word tokens for every file
    '''
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='replace') #open the file
        ftxt_unprocessed = f.read() #read the text of the file
        ftoks = preprocess_file(ftxt_unprocessed) #preprocessing the text to form word tokens
        file_tokens.append(ftoks)
    return file_tokens

def check_alnum(tok):
    '''
        Remove non-alphanumeric characters from a string
    '''
    tok = ''.join(x for x in tok if x.isalnum() == True)
    return tok
def remove_punct(tok):
    '''
        Remove the punctuation in token
    '''
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok

def remove_space(tok):
    '''
        Remove the spaces in token
    '''
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):
    '''
        Preprocess the file text and converting to word tokens
        Input: string File text
        Returns file_tokens, word tokens for the pre processed text
    '''
    #converting text to lowercase
    text = file_text.lower()

    #Fixing the contractions
    text = contractions.fix(text)

    #Performing word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = set(all_toks) #taking only the unique tokens

    #Omitting all the non-alphanumeric characters in tokens
    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]

    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        if(ctok != ""):
            toks_no_punct.append(ctok)

    #removing spaces in any remain
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        if(ctok != ""):
            cleaned_toks.append(ctok)
    cleaned_toks = list(set(cleaned_toks))
    final_tokens = [tok for tok in cleaned_toks]

    return final_tokens

def preprocess_query(query_text):
    '''
        Performs the pre processing of the query text
        Input: A string representing query text
        Output: Query word tokens
    '''

    #Performing lowercasing
    text = query_text.lower()
    
    #Fixing the contractions
    text = contractions.fix(text)

    #word tokenization
    all_toks = word_tokenize(text)

    #taking only unique tokens
    all_unique_toks = []
    for tok in all_toks:
        if(tok not in all_unique_toks):
            all_unique_toks.append(tok)

    #Omitting all the non-alphanumeric characters in tokens
    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]

    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        if(ctok != ""):
            toks_no_punct.append(ctok)

    #removing spaces in any remain
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        if(ctok != ""):
            cleaned_toks.append(ctok)
    
    #finally taking all the unique query tokens after all the preprocessing steps
    final_tokens = []
    for tok in cleaned_toks:
        if(tok not in final_tokens):
            final_tokens.append(tok)

    return final_tokens

In [6]:
#read the files to extract word tokens from each and every file
file_toks = read_files(file_paths)

In [7]:
def create_inverted_index(file_toks):
    ''' 
        This function build the inverted index. It takes in the word tokens of each file as input and returns two dictionaries: 
        inv_index_postings - corresponding to the posting list of all the terms containg the document IDs correspondg to each term and 
        inv_index_frequency - corresponding to the document frequency of each term (no. of documents containing the term).
        for each term.
    '''
    inv_index_postings = {} #this is dictionary to store the postings for the terms
    inv_index_frequency = {} #this is dictionary to store the frequency values for the terms

    #Iterate over all the files
    for i in range(len(file_toks)):
        #For each file, iterate over all the file tokens
        for tok in file_toks[i]:
            if(tok not in inv_index_postings.keys()): #if the token is not yet present as a term in the index
                inv_index_postings[tok] = [i] #create a new entry for the term in the index and intilize it with document_ID 'i'
                inv_index_frequency[tok] = 1 #create a new entry for the term in the frequency array and initialize frequency value as 1.
            else: #else if the token is already present as a term in the index
                if(i not in inv_index_postings[tok]): #if the document ID 'i' is not yet added to the posting list for the term
                    inv_index_postings[tok].append(i) #then add that
                    inv_index_frequency[tok] += 1 #also increment the frequency value by 1

    inv_index_postings = dict(sorted(inv_index_postings.items())) #sort the index in alphabetical order wrt terms
    terms_list = inv_index_postings.keys()
    for word in terms_list:
        inv_index_postings[word].sort() #sort the indivdual posting lists correponging to each term in the index
    return inv_index_postings, inv_index_frequency

In [8]:
def getDocsFromID(docID_to_docs, doc_IDs):
    '''
        Given a list of document IDs, it outputs the document names corresponding to thos IDs.
        Input: docID_to_docs (mapping between docID -> doc_name), docIDs - list of input document IDs
        Returns: doc_names - list of doc_names corresponding to document IDs in doc_IDs
    '''
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(docID_to_docs[doc_ID])
    return doc_names

In [9]:
def retreive_postings(inv_index_postings, term):
    '''
        Given a term, retreive its posting list.
        Input: inv_index_postings - inverted index postings, term
        Returns: [] if term not in index, posting list for the term otherwise
    '''
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return []
    else:
        posting = inv_index_postings[term]
        return posting

In [10]:
def retreive_frequency(inv_index_frequency, term):
    '''
        Given a term, retreive its frequency value.
        Input: inv_index_frequency - inverted index frequency array, term
        Returns: 0 if term not in index, frequency value for the term otherwise
    '''
    terms_list = inv_index_frequency.keys()
    if(term not in terms_list):
        return 0
    else:
        freq = inv_index_frequency[term]
        return freq

In [11]:
def retreive_term_info(inv_index_postings, inv_index_frequency, term):
    '''
        Given a term, retreive both its posting list & frequency value.
        Input: inv_index_postings - inverted index (posting lists for all the terms), inv_index_frequency - inverted index frequency array, term
        Returns: [], 0 if term not in index; posting, frequency value for the term otherwise
    '''
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return [], 0
    else:
        posting = inv_index_postings[term]
        freq = inv_index_frequency[term]
        return posting, freq

In [12]:
def query_AND(posting1, posting2, verbose=False):
    '''
        Implementation of the AND query
    '''
    #posting1, posting2 - posting list of the arguments of the AND operation

    #If either posting1 or posting2 is empty their AND will return an empty list
    if(posting1 == [] or posting2 == []):
        return 0, 0, [], []

    #We will iterate over both the posting lists and perform their intersection by using a merging based algorithm
    ptr1 = 0 #pointer that iterates over posting1
    ptr2 = 0 #pointer that iterates over posting2
    answer_docID = [] #contains the answer

    num_comparisons = 0 #intialize number of comparison as 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2)): #checking that both pointers remain in range
        num_comparisons += 1 #increment the number of comparisons by one
        if(posting1[ptr1] == posting2[ptr2]):
            #when the the value pointed by ptr1 and ptr2 is same, means that this is a common document ID 
            #present in both posting1 and positng2 and henche will be included in our answer.
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        #otherwise move the pointer pointing to a smaller value forward by one
        elif(posting1[ptr1] < posting2[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID) #number of docs retreived
    doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID) #names of docs retreived

    if(verbose==True):
        print(f"AND Query: \nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")

    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR(posting1, posting2, verbose=False):
    '''
        Implementation of the OR query
    '''

    #posting1, posting2 - posting list of the arguments of the OR operation

    #if both posting1 and posting2 are empty lists, their OR i.e a  kind of union will be empty
    if((posting1 == []) and (posting2 == [])):
        return 0, 0, [], []
    #else if posting1 is empty but posting2 is not, then their OR i.e a  kind of union will only contain all values of posting2
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = posting2
        return len(ans_docs), 0, getDocsFromID(docID_to_doc_mapping, ans_docs), ans_docs
    #else if posting2 is empty but posting1 is not, then their OR i.e a  kind of union will only contain all values of posting1
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, getDocsFromID(docID_to_doc_mapping, ans_docs), ans_docs
    #else when none of them are empty
    else:

        #We will iterate over both the posting lists and perform their union by using a merging based algorithm
        ptr1 = 0 #pointer that iterates over posting1
        ptr2 = 0 #pointer that iterates over posting2
        answer_docID = [] #contains the answer

        num_comparisons = 0 #intialize number of comparison as 0
        
        while(ptr1 < len(posting1) and ptr2 < len(posting2)): #checking that both pointers remain in range
            num_comparisons += 1 #increment the number of comparisons by 1
            if(posting1[ptr1] == posting2[ptr2]):
            #when the the value pointed by ptr1 and ptr2 is same, means that this is a common document ID 
            #present in both posting1 and positng2 and hence it will be included only once in our answer.
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            #otherwise include the document ID having lower magnitude and move the pointer pointing to a smaller value forward by one
            elif(posting1[ptr1] < posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2[ptr2])
                ptr2 += 1
        #If the posting1 list has not been completly iterated, finish iterating it anc include all the values remaining in that in our answer
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        #If the posting2 list has not been completly iterated, finish iterating it anc include all the values remaining in that in our answer
        while(ptr2 < len(posting2)):
            answer_docID.append(posting2[ptr2])
            ptr2 += 1

        num_docs_retreived = len(answer_docID) #number of docs retreived
        doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID) #names of docs received

        if(verbose==True):
            print(f"Query OR\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def perform_NOT(posting):
    '''
        Perform the NOT operation given a posting list
    '''
    #Here we simply return all the valid values(/docIDs) that are not present the input posting list
    all_docIDs = [docID for docID in range(len(file_names))]
    if(posting == []): #if the input posting list is empty  return all document IDs
        return all_docIDs
    for docID in posting: #else remove the document ID from the universal set
        all_docIDs.remove(docID)
    return all_docIDs

def query_AND_NOT(posting1, posting2, verbose=False):
    '''
        Perform AND NOT query
    '''

    if(posting1 == []): #if posting1 is empty, then it's AND with anything will be empty
        return 0, 0, [], []
    
    ptr1 = 0 #pointer that iterates over posting1
    ptr2 = 0 #pointer that iterates over posting2
    answer_docID = [] #stores the answers

    num_comparisons = 0 #initialize number of comparisons as zero

    while(ptr1 < len(posting1) and ptr2 < len(posting2)): #checking that both pointers remain in range
        num_comparisons += 1 #increment the number of comparisons by 1
        if(posting1[ptr1] == posting2[ptr2]):
            #when the the value pointed by ptr1 and ptr2 is same, means that this is a common document ID 
            #present in both posting1 and positng2 and hence it will not be included in our answer.
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2[ptr2]): #if ptr1 lags behind ptr2 then we have to add the element of posting1 to our answer and increment ptr1
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        else: #else do not add anything to answer and increment ptr2
            ptr2 += 1
    while(ptr1 < len(posting1)): #if anything is left in posting1 then add it till ptr1 reaches the end
        answer_docID.append(posting1[ptr1])
        ptr1 += 1
    
    num_docs_retreived = len(answer_docID) #number of docs retreived
    doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID) #names of docs retreived

    if(verbose==True):
        print(f"Query AND NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR_NOT(posting1, posting2, verbose=False):
    '''
        Perform OR NOT query
    '''
    #Here we follow the approach that to that to do: x OR NOT y, we first calculate NOT y and then calculate its OR with x.

    posting2_NOT = perform_NOT(posting2) #perform NOT of posting2

    #if both posting1 and posting2_NOT are empty lists, their OR i.e a  kind of union will be empty
    if((posting1 == []) and (posting2_NOT == [])):
        return 0, 0, [], []
    #else if posting1 is empty but posting2_NOT is not, then their OR i.e a  kind of union will only contain all values of posting2_NOT
    elif((posting1 == []) and (posting2_NOT != [])):
        ans_docs = posting2_NOT
        return len(ans_docs), 0, getDocsFromID(docID_to_doc_mapping, ans_docs), ans_docs
    #else if posting2_NOT is empty but posting1 is not, then their OR i.e a  kind of union will only contain all values of posting1
    elif((posting1 != []) and (posting2_NOT == [])):
        ans_docs = posting1
        return len(ans_docs), 0, getDocsFromID(docID_to_doc_mapping, ans_docs), ans_docs
    else:
        
        #now we will perform AND of posting1 and posting2_NOT. We will iterate over both the posting lists and perform their union by using a merging based algorithm
        ptr1 = 0 #pointer that iterates over posting1
        ptr2 = 0 #pointer that iterates over posting2_NOT
        answer_docID = [] #contains the answer(docIDs)
        num_comparisons = 0 #initialize number of comparisons as 0

        while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)): #checking that both pointers remain in range
            num_comparisons += 1 #increment the number of comparisons by one
            if(posting1[ptr1] == posting2_NOT[ptr2]):
                #when the the value pointed by ptr1 and ptr2 is same, means that this is a common document ID 
                #present in both posting1 and positng2 and hence it will be included only once in our answer.
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            #otherwise include the document ID having lower magnitude and move the pointer pointing to a smaller value forward by one
            elif(posting1[ptr1] < posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2_NOT[ptr2])
                ptr2 += 1
        #If the posting1 list has not been completly iterated, finish iterating it and include all the values remaining in that in our answer
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        #If the posting2_NOT list has not been completly iterated, finish iterating it anc include all the values remaining in that in our answer
        while(ptr2 < len(posting2_NOT)):
            answer_docID.append(posting2_NOT[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID) #number of docs retreived
        doc_names_retreived = getDocsFromID(docID_to_doc_mapping, answer_docID) #names of docs retreived

        if(verbose==True):
            print(f"Query OR NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

In [20]:
def process_ops(ops_string):
    '''
        This function processes the input operation sequence and converts that into an array containing the operators that have to be applied
    '''
    ops = ops_string.split(",")
    ops = [op.strip() for op in ops]
    for op in ops:
        if(not(op == "AND" or op == "OR" or op == "OR NOT" or op == "AND NOT")):
            return []
    return ops

def perform_query(posting1, posting2, op):
    '''
        Given the postings and an operator(string op), call this function calls the appropriate operation(AND/OR/AND NOT/OR NOT)
    '''
    if(op == "AND"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "AND NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    else:
        print("INVALID OPERATOR")
        return []

def process_query(inv_index, query_tokens, ops):
    '''
        This function processes the whole query sequentially.
    '''
    n_qtoks = len(query_tokens)
    n_ops = n_qtoks - 1 #setting the needed number of operators as number of query tokens minus one
    if(len(ops) > n_ops): #if there are more operators than needed
        print(f"Insufficient tokens in the query / Excess operators")
        print("\n----------------------------------------------\n")
        return -1, -1, []
    elif(len(ops) < n_ops): #if there are less operators than needed
        print(f"Insufficient operators / Excess query tokens")
        print("\n----------------------------------------------\n")
        return -1, -1, []
    else:
        total_comparisons = 0 #initiliazing the total number of comparisons for the whole query as zero
        if(n_ops == 1): #if there is only one operator
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index,query_tokens[1])
            num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = perform_query(p1, p2, ops[0])
            return num_docs_retreived, num_comparisons, doc_names_retreived
        else:
            #sequentially keep on processing the operations given
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index, query_tokens[1])
            ndocs, total_comparisons, doc_names, ans = perform_query(p1, p2, ops[0])
            qtok_ptr = 2
            for i in range(1, len(ops)): #sequentially take up each operator and perform the operation between previous operations answer and the next token
                posting_tok = retreive_postings(inv_index, query_tokens[qtok_ptr])
                ndocs, cmps, doc_names, ans = perform_query(ans, posting_tok, ops[i])
                total_comparisons += cmps #increment the total number of comparisons
                qtok_ptr += 1
            num_docs_retreived = len(ans) #number of documents finally retreived
            doc_names_retreived = getDocsFromID(docID_to_doc_mapping, ans) #names of the documents finally retreived
            return num_docs_retreived, total_comparisons, doc_names_retreived

In [14]:
inverted_index_postings, inverted_index_frequency = create_inverted_index(file_toks) #creating the inverted index and inverted_index_frequency dictionary

In [24]:
num_queries = int(input("Enter the number of queries you want to run : "))
for i in range(num_queries):
    input_query_sequence = input("Enter the space-separated query terms : ")
    input_operator_sequence = input("Enter the comma-separated operator sequence enclosed in '[]' : ")
    print(f"Input query sequence : {input_query_sequence}")
    print(f"Input operator sequence : {input_operator_sequence}")
    input_operators = process_ops(input_operator_sequence[1:len(input_operator_sequence) - 1])
    query_tokens = preprocess_query(input_query_sequence)
    print(f"Input operator (array) : {input_operators}")
    print(f"Preprocessed query tokens : {query_tokens}\n")
    query_result_ndocs, query_result_total_cmps, query_result_doc_names = process_query(inverted_index_postings, query_tokens, input_operators)
    
    if(query_result_ndocs != -1):
        query_generated = query_tokens[0] + " " + input_operators[0] + " " + query_tokens[1] + " "
        ptr_qtoks = 2
        for i in range(1, len(input_operators)):
            query_generated += input_operators[i] + " "
            query_generated += query_tokens[ptr_qtoks] + " "
            ptr_qtoks += 1
        print(f"Query : {query_generated}\n")
        print(f"No. of documents retreived: {query_result_ndocs}\nNumber of comparisons: {query_result_total_cmps}\nNames of retreived documents: {query_result_doc_names}")
        print("\n----------------------------------------------\n")

Input query sequence : deem lion
Input operator sequence : [OR]
Input operator (array) : ['OR']
Preprocessed query tokens : ['deem', 'lion']

Query : deem OR lion 

No. of documents retreived: 23
Number of comparisons: 19
Names of retreived documents: ['bitnet.txt', 'boneles2.txt', 'booze1.fun', 'christop.int', 'collected_quotes.txt', 'computer.txt', 'deep.txt', 'dromes.txt', 'dthought.txt', 'filmgoof.txt', 'japantv.txt', 'lion.jok', 'lion.txt', 'lions.cat', 'llong.hum', 'murphys.txt', 'murphy_l.txt', 'onetotwo.hum', 'pracjoke.txt', 'puzzles.jok', 'stuf10.txt', 'three.txt', 'tpquotes.txt']

----------------------------------------------

